In [ ]:
import sagemaker, boto3, json

from sagemaker.estimator import Estimator
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

import pandas as pd

from sklearn.model_selection import train_test_split

import sys
import importlib
import uuid

In [18]:
import my_config
importlib.reload(my_config)

<module 'my_config' from 'c:\\Users\\XMF4277_1\\Documents\\repos\\aws\\arch_sagemaker_lambda_api-gateway\\notebooks\\my_config.py'>

In [19]:
boto3_session = boto3.Session()
sm_session = sagemaker.Session()

aws_region = boto3_session.region_name
bucket_name = my_config.bucket_name

print(aws_region)
print(sm_session)
print(bucket_name)
print(my_config.my_aws_role)

us-east-1
sagemaker-api-lambda
arn:aws:iam::007863746889:role/sagemaker-api-lambda-role


In [20]:
MODEL_ID, MODEL_VERSION = my_config.model_id, "3.0.15"
TRAINING_INSTANCE_TYPE = my_config.training_instance_type

train_source_uri = script_uris.retrieve(
    model_id=MODEL_ID, model_version=MODEL_VERSION, script_scope="training"
)

train_model_uri = model_uris.retrieve(
    model_id=MODEL_ID, model_version=MODEL_VERSION, model_scope="training"
)

train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=MODEL_ID,
    model_version=MODEL_VERSION,
    image_scope="training",
    instance_type=TRAINING_INSTANCE_TYPE
)

In [21]:
hyperparameters_model = hyperparameters.retrieve_default(model_id=MODEL_ID, model_version=MODEL_VERSION)

hyperparameters_model["batch_size"] = "32"
hyperparameters_model["learning_rate"] = '1e-6'
hyperparameters_model["epochs"] = 1

hyperparameters_model

{'train_only_top_layer': 'False',
 'epochs': 1,
 'batch_size': '32',
 'optimizer': 'adamw',
 'learning_rate': '1e-6',
 'warmup_steps_fraction': '0.1',
 'beta_1': '0.9',
 'beta_2': '0.999',
 'momentum': '0.9',
 'epsilon': '1e-06',
 'rho': '0.95',
 'initial_accumulator_value': '0.1',
 'early_stopping': 'False',
 'early_stopping_patience': '5',
 'early_stopping_min_delta': '0.0',
 'dropout_rate': '0.2',
 'regularizers_l2': '0.01',
 'validation_split_ratio': '0.2',
 'reinitialize_top_layer': 'Auto'}

In [22]:
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_prefix = "training-datasets/SST/"

training_data_s3path = f"s3://{training_data_bucket}/{training_data_prefix}"

ouput_bucket = bucket_name
ouput_prefix = "TC"

s3_output_location = f"s3://{ouput_bucket}/{ouput_prefix}/output"
s3_output_location

's3://sagemaker-api-lambda/TC/output'

In [23]:
!aws s3 cp --recursive $training_data_s3path data/sst2

Completed 256.0 KiB/3.8 MiB (246.6 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/3.8 MiB (439.5 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/3.8 MiB (627.8 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/3.8 MiB (830.1 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/3.8 MiB (997.3 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/3.8 MiB (1.2 MiB/s) with 1 file(s) remaining    
Completed 1.8 MiB/3.8 MiB (1.3 MiB/s) with 1 file(s) remaining    
Completed 2.0 MiB/3.8 MiB (1.5 MiB/s) with 1 file(s) remaining    
Completed 2.2 MiB/3.8 MiB (1.7 MiB/s) with 1 file(s) remaining    
Completed 2.5 MiB/3.8 MiB (1.8 MiB/s) with 1 file(s) remaining    
Completed 2.8 MiB/3.8 MiB (2.0 MiB/s) with 1 file(s) remaining    
Completed 3.0 MiB/3.8 MiB (2.2 MiB/s) with 1 file(s) remaining    
Completed 3.2 MiB/3.8 MiB (2.3 MiB/s) with 1 file(s) remaining    
Completed 3.5 MiB/3.8 MiB (2.5 MiB/s) with 1 file(s) remaining    
Completed 3.8 MiB/3.8 MiB (2.7 MiB/s) with 1 file(s) remaining

In [24]:
training_data_df = pd.read_csv("data/sst2/data.csv", header=None)
training_data_df.columns = ["label", "text"]

training_data_df.head()

,label,text
0,0,hide new secretions from the parental units
1,0,"contains no wit , only labored gags"
2,1,that loves its characters and communicates som...
3,0,remains utterly satisfied to remain the same t...
4,0,on the worst revenge-of-the-nerds clichés the ...


In [25]:
training_data_df.shape

(68221, 2)

In [26]:
# Using only a fraction to train because computational power
train_df, test_df = train_test_split(training_data_df, train_size=0.01, random_state=43)

In [27]:
train_df.head()

,label,text
34863,1,"venturesome , beautifully realized"
12635,0,of self-congratulation between actor and direc...
50129,0,a pact to burn the negative and the script and...
40766,1,-- conrad l. hall 's cinematography will likel...
34223,0,that it makes your least favorite james bond m...


In [28]:
train_df.to_csv("data/sst2/train.csv", header=False, index=False)

In [29]:
# Uploading local file to s3
prefix_s3 = "TC"
target_s3_file_path = "train/data.csv"

local_train_file_path = "data/sst2/train.csv"

s3_full_path_target = f"{prefix_s3}/{target_s3_file_path}"

boto3_session.resource("s3").Bucket(bucket_name).Object(s3_full_path_target).upload_file(local_train_file_path)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [ ]:
training_job_name = f"{my_config.solution_prefix}-tc-finetune"

tc_estimator = Estimator(
    role=my_config.my_aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=TRAINING_INSTANCE_TYPE,
    max_run=3500,
    hyperparameters=hyperparameters_model,
    output_path=s3_output_location,
    tags=[{"Key": my_config.tag_key, "Value": my_config.solution_prefix}],
    base_job_name=training_job_name
)

In [31]:
training_data_path_updated = f"s3://{bucket_name}/{prefix_s3}/train"
training_data_path_updated

's3://sagemaker-api-lambda/TC/train'

In [32]:
tc_estimator.fit({"training": training_data_path_updated}, logs=True)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-soln-documents--tc-finetune-2025-06-19-18-38-02-665


2025-06-19 18:38:05 Starting - Starting the training job...
2025-06-19 18:38:21 Starting - Preparing the instances for training...
2025-06-19 18:39:11 Downloading - Downloading the training image......
2025-06-19 18:39:58 Training - Training image download completed. Training in progress.2025-06-19 18:40:05.236392: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-06-19 18:40:05.236548: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-06-19 18:40:05.261177: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-06-19 18:40:07,186 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-06-19 18:40:07,198 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-19 1

In [47]:
INFERENCE_INSTANCE_TYPE = my_config.inference_instance_type

deply_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=MODEL_ID,
    model_version=MODEL_VERSION,
    instance_type=INFERENCE_INSTANCE_TYPE
)

deploy_source_uri = script_uris.retrieve(
    model_id=MODEL_ID, model_version=MODEL_VERSION, script_scope="inference"
)

unique_hash = str(uuid.uuid4())[:6]
endpoint_name_tc_finetune = f"{my_config.solution_prefix}-{unique_hash}-tc-finetune-endpoint"

In [56]:
finetuned_predictor = tc_estimator.deploy(

    entry_point="inference.py",
    image_uri=deply_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name_tc_finetune,
    serverless_inference_config=ServerlessInferenceConfig()

)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-007863746889
INFO:sagemaker:Repacking model artifact (s3://sagemaker-api-lambda/TC/output/sagemaker-soln-documents--tc-finetune-2025-06-19-18-38-02-665/output/model.tar.gz), script artifact (s3://jumpstart-cache-prod-us-east-1/source-directory-tarballs/tensorflow/inference/tc/v2.0.0/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-007863746889/sagemaker-jumpstart-2025-06-19-19-27-14-161/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2025-06-19-19-27-14-161
INFO:sagemaker:Creating endpoint-config with name sagemaker-soln-documents--0e0686-tc-finetune-endpoint
INFO:sagemaker:Creating endpoint with name sagemaker-soln-documents--0e0686-tc-finetune-endpoint


--!

In [76]:
inference_data_endpoint = test_df[:32]

ground_truth, test_examples = (
    inference_data_endpoint.iloc[:, 0].values.tolist(),
    inference_data_endpoint.iloc[:, 1].values.tolist()
)

In [77]:
def query_endpoint(text, predictor):
    response = predictor.predict(
        text,
        {"ContentType": "application/x-text", "Accept": "application/json;verbose"}
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response)
    probabilities, labels, predicted_label = (
        model_predictions["probabilities"],
        model_predictions["labels"],
        model_predictions["predicted_label"]
    )
    return probabilities, labels, predicted_label

In [ ]:
predict_prob_list, predict_label_list = list(), list()

for text in test_examples:
    
    query_response = query_endpoint(text.encode("utf-8"), finetuned_predictor)
    probabilities, labels, predicted_labels = parse_response(query_response)  
    
    predict_prob_list.append(probabilities)
    predict_label_list.append(predicted_labels)

Configurar función lambda en aws (habilita permisos con sagemaker para invocar endpoints) para invocar el endpoint del modelo. Configurar las variables de entorno pertinentes

In [ ]:
import os
import json
import boto3

# Grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] # Configurado en variables de ambiente del AWS
runtime = boto3.client('runtime.sagemaker') # Asegurarse de que la función tiene permisos de invocación de endpoints

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    # Assuming the input is in the format {'text': 'your text here'}
    data = json.loads(json.dumps(event))
    text = data['text']
    
    # Encode the text
    encoded_text = text.encode('utf-8')
    
    # Query the endpoint
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/x-text',
                                       Accept='application/json;verbose',
                                       Body=encoded_text)

    print("Raw response:", response)

    # Parse the response
    response_body = json.loads(response['Body'].read().decode())
    print("Parsed response:", response_body)

    probabilities = response_body['probabilities']
    labels = response_body['labels']
    predicted_label = response_body['predicted_label']
    
    # Convert predicted_label to 'Positive' or 'Negative'
    sentiment = 'Positive' if predicted_label == 1 else 'Negative'
    
    result = {
        'sentiment': sentiment,
        'probabilities': probabilities,
        'labels': labels
    }
    
    return result


Crear una API REST (APY-GATEWAY) para comunicarse entre el cliente, con la función y lambda y luego la función con el endpoint. Crear método referenciando a la lambda creada y hacer un deploy del mismo

Se hace un request a la API usando el URI

In [107]:
import requests

api_uri = "https://9awwa68h1k.execute-api.us-east-1.amazonaws.com/DEPLOYMENT_ENV"
resource_path = "text-clasification-funetune-model"

response = requests.post(
    f"{api_uri}/{resource_path}",
    data=json.dumps({"text": "I loved this movie. It was amazing"})
)

response

<Response [200]>

In [108]:
response.content

b'{"sentiment": "Positive", "probabilities": [0.0652378, 0.93476218], "labels": [0, 1]}'

In [109]:
# Liberamos el modelo y el endpoint para no incurrir en posibles gastos (aunque fue severless)
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-jumpstart-2025-06-19-19-27-14-161
INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-soln-documents--0e0686-tc-finetune-endpoint
INFO:sagemaker:Deleting endpoint with name: sagemaker-soln-documents--0e0686-tc-finetune-endpoint
